# NEXT25 BRK2-187: Secure your cloud with Google Cloud’s security and compliance innovations

## What You'll learn

1.  Google Cloud’s security building blocks and how they fit together to provide a foundation for your workloads
2.  How to meet baseline compliance for a real application and it’s data
3.  How to protect sensitive PII with strong controls without changing the workload
4.  How to create guardrails, while enabling developers to self-serve


## Demo: Assured Workload Migration API

Parsing via jq generated by Gemini interaction

In [ ]:
%%bash

DESTINATION_ORGANIZATION_ID="YOUR_ORG"
DESTINATION_WORKLOAD_ID="YOUR_ASSURED_WORKLOAD_ID"
SOURCE_PROJECT_ID="YOUR_PROJECT"
QUOTA_PROJECT_ID="QUOTA_PROJECT"
REGION="us-east4"

export GOOGLE_CLOUD_QUOTA_PROJECT=$QUOTA_PROJECT_ID
gcloud config set project $QUOTA_PROJECT_ID

#Might need to do 'gcloud autho login' after connection
curl -s -X GET      -H "Authorization: Bearer $(gcloud auth print-access-token)"      -H "x-goog-user-project: $SOURCE_PROJECT_ID"      "https://$REGION-assuredworkloads.googleapis.com/v1/organizations/$DESTINATION_ORGANIZATION_ID/locations/$REGION/workloads/$DESTINATION_WORKLOAD_ID:analyzeWorkloadMove?project=projects/$SOURCE_PROJECT_ID" | jq -r '
  .assetMoveAnalyses[] as $asset | # Iterate through each asset analysis, store current asset in $asset variable
  $asset.analysisGroups[]?.analysisResult | # Navigate into analysisGroups (if it exists) and then analysisResult
  ( # Start a block to handle warnings and blockers separately but combine their outputs
    (.warnings[]? | {type: "Warning", detail: .detail}), # Process warnings: create an object for each warning
    (.blockers[]? | {type: "Blocker", detail: .detail})  # Process blockers: create an object for each blocker (if any)
  ) | # Pipe the resulting {type, detail} objects
  "\u001b[31m\(.type)\u001b[0m \(.detail) \n\u001b[1m\($asset.asset)\u001b[0m\n" # Format the output string: Type, Detail, Resource Name
' -

## Demo: Confidential Mode for Hyperdisk

In [ ]:
%%bash

PROJECT_ID="PROJECT_ID"
SNAPSHOT_NAME="SNAPSHOT_NAME"
ZONE="europe-west4-b"
CHD_NAME="next-demo-chd"
KMS_NAME="FULL_KEY_NAME"

gcloud compute disks create $CHD_NAME \
     --zone=$ZONE \
     --source-snapshot=$SNAPSHOT_NAME \
     --type=hyperdisk-balanced \
     --kms-key=$KMS_NAME \
     --confidential-compute

## Demo: Confidential Dataflow

Follow instructions for Dataflow flex template "getting started"
https://cloud.google.com/dataflow/docs/guides/templates/using-flex-templates#python


In [ ]:
%%bash

BUCKET_NAME="eubanking"
LOCATION="europe-west4"

gcloud dataflow flex-template run "client-bank-cvm-`date +%Y%m%d-%H%M%S`" \
 --template-file-gcs-location "gs://$BUCKET_NAME/client_info.json" \
 --parameters output="gs://$BUCKET_NAME/processed-client-cards-" \
 --region "$LOCATION" \
 --disable-public-ips \
 --network="next-eubanking" \
 --subnetwork="regions/europe-west4/subnetworks/next-eubanking-sn" \
 --additional-experiments=enable_confidential_compute


## Demo: Custom Org policies
Create a machine policy

In [ ]:
%%bash

PROJECT_ID="PROJECT_ID"
ORG_ID="ORD_ID"
ZONE="europe-west4-a"

cat <<EOF > ./custom_constraint_1.yaml
name: organizations/$ORD_ID/customConstraints/custom.restrictInstanceMachineType
resourceTypes:
- compute.googleapis.com/Instance
methodTypes:
- CREATE
condition: resource.machineType.contains('n2') || resource.machineType.contains('e2')
actionType: ALLOW
displayName: Restrict GCE VM Instances to N2 or E2
description: N2 and E2 instances are permitted in this project. Please select a different machine type or contact susan@helpdesk.
EOF

gcloud org-policies set-custom-constraint ./custom_constraint_1.yaml --project $PROJECT_ID

cat <<EOF > ./enforce_constraint_1.yaml
name: projects/$PROJECT_ID/policies/custom.restrictInstanceMachineType
spec:
  rules:
  - enforce: true
EOF

gcloud org-policies set-custom-constraint ./custom_constraint_1.yaml

gcloud compute instances create this-vm-is-blocked --project=$PROJECT_ID --zone=$ZONE --machine-type=m3-ultramem-32 --network-interface=network=default

gcloud compute instances create this-vm-is-allowed --project=$PROJECT_ID --zone=$ZONE --machine-type=n2-standard-2 --network-interface=network=default
